<a href="https://colab.research.google.com/github/Adyypower/LLMs-Model/blob/main/ReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install necessary libraries
!pip install -qU langchain langchain_google_genai langgraph

# 2. Set up your Google API Key
import os
from google.colab import userdata

# Make sure you have created a secret named "GOOGLE_API_KEY" in your Colab environment
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [2]:
# 3. Import required modules from LangChain and LangGraph
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage , ToolMessage , SystemMessage
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode



In [3]:
class AgentState(TypedDict):
  messages:Annotated[Sequence[BaseMessage],add_messages]

In [4]:
@tool

def add(a:int,b:int):
  """ this is an additin function that add 2 numberr together"""
  return a+b

In [15]:
@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b


In [16]:
@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

In [32]:
tools = [add,subtract,multiply]

In [26]:
model= ChatGoogleGenerativeAI(model="gemini-2.5-flash").bind_tools(tools)

In [27]:
def model_call(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content=
        "You are my AI assistant, please answer my query to the best of your ability."
    )
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}

In [28]:
def should_continue(state:AgentState)->AgentState:
  messages = state["messages"]
  last_message = messages[-1]
  if not last_message.tool_calls:
    return "end"
  else:
    return "continue"

In [29]:
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)

In [33]:
tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

In [30]:
graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)


In [34]:
graph.add_edge("tools","our_agent")
app = graph.compile()

In [13]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        message.pretty_print()


In [35]:
from langchain_core.messages import HumanMessage

inputs = {
    "messages": [
        HumanMessage(content="Add 40 + 12 and then multiply result by 6 then also tell me a joke please.")
    ]
}

print_stream(app.stream(inputs, stream_mode="values"))


================================ Human Message =================================

Add 40 + 12 and then multiply result by 6 then also tell me a joke please.
================================== Ai Message ==================================
Tool Calls:
  add (99d6ef70-15a9-4abe-9504-34d5241f4c7f)
 Call ID: 99d6ef70-15a9-4abe-9504-34d5241f4c7f
  Args:
    a: 40
    b: 12
================================= Tool Message =================================
Name: add

52
================================== Ai Message ==================================
Tool Calls:
  multiply (abde0eeb-84a6-4e7d-84fa-00bb12a2a76b)
 Call ID: abde0eeb-84a6-4e7d-84fa-00bb12a2a76b
  Args:
    a: 52
    b: 6
================================= Tool Message =================================
Name: multiply

312
================================== Ai Message ==================================

The sum of 40 and 12 is 52. When you multiply 52 by 6, the result is 312.

Here's a joke for you: Why don't scientists trust atoms? Bec